# Whole brain data processing

In [10]:
import numpy as np
import zarr
import shutil
import os, sys
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from cellProcessing import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Set up environments

* Install `fish_proc` from github https://github.com/zqwei/fish_processing

In [2]:
## STEP 1: specify data and save path
dir_root = '/groups/ahrens/ahrenslab/jing/giving_up/20190219/fish2/7dpf-huc_gcamp7ff_gfap_rgeco-GU_slow_fwd-fish02-exp02_20190219_174013/im'
# save_root = '/nrs/ahrens/jing/20190219/fish2/7dpf-huc_gcamp7ff_gfap_rgeco-GU_slow_fwd-fish02-exp02_20190219_174013/processed'

### Set save folder to Ziqiang's folder

In [3]:
save_root = './processed'
if not os.path.exists(save_root):
    os.makedirs(save_root)

## Preproecssing
This will generate the follow files at `save_root` folder
* `motion_fix_.h5` -- reference image
* `trans_affs.npy` -- affine transform
* `Y_2dnorm_ave.h5` -- average image after detrend
* `local_pca_data.zarr` -- local pca denoised image (This might be used a x4 file size comparing to the raw data), which can be removed afterwards

In [ ]:
%%time
baseline_percentile = 20  
baseline_window = 1000   # number of frames
numCores = 200  # requests for dask workers
preprocessing(dir_root, save_root, numCores=numCores, window=baseline_window, percentile=baseline_percentile)

## Data transpose and rechunk

In [4]:
if os.path.exists(f'{save_root}/motion_corrected_data.zarr'):
    trans_data_ = zarr.open(f'{save_root}/motion_corrected_data.zarr', mode='r')
    trans_data_t = zarr.open(f'{save_root}/motion_corrected_data_by_t.zarr', mode='r')
    t, z, x, y = trans_data_.shape
    z_, x_, y_, t_ =trans_data_t.shape
    if t==t_ and z==z_ and x==x_ and y==y_:
        print('Shape of arrays are correct, continue--')
    nz = np.random.randint(z)
    nt = np.random.randint(t)
    if np.array_equal(trans_data_[nt, nz], trans_data_t[nz, :, :, nt]):
        shutil.rmtree(f'{save_root}/motion_corrected_data.zarr')

In [ ]:
if np.array_equal(trans_data_[nt, nz], trans_data_t[nz, :, :, nt]):
    shutil.rmtree(f'{save_root}/motion_corrected_data.zarr')

## Local PCA
* request a single machine with 32 or 48 cores

In [ ]:
trans_data_t = da.from_zarr(f'{save_root}/motion_corrected_data_by_t.zarr')
chunks = trans_data_t.chunksize

## Mask
Making a mask for data to decrease the number of computation in demix
* remove low intensity pixel
* remove low snr pixel
* remove low local pixel correlation pixels

This will generate the follow files at `save_root` folder
* `mask_map.h5` -- mask
* `local_correlation_map.h5` -- local correlation
* `masked_local_pca_data.zarr` -- df/f computation
* `masked_downsampled_local_pca_data.zarr` -- downsampled in time -> used for cell segmentation, which can be removed afterwards

In [ ]:
## mask parameters
intensity_percentile = 40  ## throws out low intensity pixels to create mask: try 20-40
dt = 5  # time downsample for cell segmentation
mask_brain(save_root, percentile=intensity_percentile, dt=dt, numCores=20, is_skip_snr=True, save_masked_data=True)

### Check mask

In [ ]:
## Check if mask doesn't look good, choose another value
# percentile is applied on a plane by plane basis (i.e. each plane has the different thresholds)
mask = File(f'{save_root}/mask_map.h5', 'r')['default'].value

In [ ]:
for n_ave_ in mask.squeeze():
    plt.imshow(n_ave_)
    plt.show()

## Demix
This will generate the follow files at `save_root` folder
* `demix_rlt` for each block

In [ ]:
nsplit = 8 # number of split on x and y -- about 256x256 is good
demix_cells(save_root, nsplit = nsplit, numCores = 200)

### Check demix results

In [ ]:
block_id = (0, 2, 0, 0)
check_demix_cells(save_root, block_id, nsplit=8, plot_global=True)

## Compute df/f
One of three choices
* df is from NMF components (where background is not included) -- preferred
* df is on raw data based on the recomputation of cell F using NMF weights
* df/f on pixels without using NMF weights

### df/f on pixels

In [ ]:
baseline_percentile = 20
baseline_window = 1000
numCores = 200
dff = compute_cell_dff_pixels(dir_root, save_root, 
                              numCores=numCores, 
                              window=baseline_window, 
                              percentile=baseline_percentile)

### df/f on raw cell data

In [ ]:
compute_cell_dff_raw(dir_root, save_root, 
                     numCores=numCores, 
                     window=baseline_window, 
                     percentile=baseline_percentile, 
                     nsplit=nsplit)

### df/f on denoised cell data

In [ ]:
compute_cell_dff_NMF(dir_root, save_root, 
                     numCores=numCores, 
                     window=baseline_window, 
                     percentile=baseline_percentile, 
                     nsplit=nsplit, dt=dt)